<a href="https://colab.research.google.com/github/francji1/01RAD/blob/main/python/01RAD_Ex3_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01RAD Exercise 03

###  0. Load the dataset and initial exploration
In Python, we'll use the statsmodels library to load the cars dataset and use pandas for data manipulation and exploration.

Q1. Display dataset (scatter plot, histograms and density plots of speed and dist variables)
For visualizations, we'll primarily use matplotlib and seaborn.



In [ ]:

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

In [ ]:

# Load the dataset
cars = sm.datasets.get_rdataset("cars").data

# Display the head of the dataset
head_cars = cars.head()
print(head_cars)
# Display dataset structure
str_cars = cars.dtypes
print(str_cars)





### 1) Display data set (scatter plot, histograms and density plots of speed and dist variables)


In [ ]:
# Display summary of the dataset
summary_cars = cars.describe()
summary_cars


In [ ]:

# Scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=cars['speed'], y=cars['dist'])
plt.title('Speed vs Stopping Distances of Cars')
plt.xlabel('Speed')
plt.ylabel('Stopping Distance')
plt.show()



In [ ]:
# Histograms and Density Plots

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histogram for speed
sns.histplot(cars['speed'], kde=False, color='red', ax=axes[0, 0])
axes[0, 0].set_title('Histogram for Speed')
axes[0, 0].set_xlabel('Speed (mph)')
axes[0, 0].set_ylabel('Count')

# Density plot for speed
sns.kdeplot(cars['speed'], color='red', ax=axes[0, 1])
axes[0, 1].set_title('Density Plot for Speed')
axes[0, 1].set_xlabel('Speed (mph)')
axes[0, 1].set_ylabel('Density')

# Histogram for dist
sns.histplot(cars['dist'], kde=False, color='blue', ax=axes[1, 0])
axes[1, 0].set_title('Histogram for Stopping Distance')
axes[1, 0].set_xlabel('Stopping Distance (ft)')
axes[1, 0].set_ylabel('Count')

# Density plot for dist
sns.kdeplot(cars['dist'], color='blue', ax=axes[1, 1])
axes[1, 1].set_title('Density Plot for Stopping Distance')
axes[1, 1].set_xlabel('Stopping Distance (ft)')
axes[1, 1].set_ylabel('Density')

plt.tight_layout()
plt.show()


In [ ]:
# Scatter plot
plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
sns.scatterplot(data=cars, x='speed', y='dist')
plt.title("Scatter plot of Speed vs Stopping Distance")

# Histograms and density plots
plt.subplot(1, 3, 2)
#sns.histplot(simulated_cars['speed'], kde=True, color='blue', label='Speed')
sns.histplot(cars['dist'], kde=True, color='red', label='Stopping Distance')
plt.legend()
plt.title("Histogram and Density Plot of Speed and Stopping Distance")

# Histograms and density plots
plt.subplot(1, 3, 3)
sns.histplot(cars['speed'], kde=True, color='blue', label='Speed')
#sns.histplot(simulated_cars['dist'], kde=True, color='red', label='Stopping Distance')
plt.legend()
plt.title("Histogram and Density Plot of Speed and Stopping Distance")

plt.tight_layout()
plt.show()


In [ ]:
# Pair plot for the  cars dataset
sns.pairplot(cars)
plt.suptitle('Pair Plot for Cars Dataset', y=1.02)
plt.show()


### 2) Try model with and without intercept


In [ ]:

# Model without intercept
model_without_intercept = smf.ols('dist ~ speed - 1', data=cars).fit()

# Model with intercept
model_with_intercept = smf.ols('dist ~ speed', data=cars).fit()

model_without_intercept.summary(), model_with_intercept.summary()



### 3) Compute "manually" OLS estimate of regression parameters and error variance


In [ ]:
# 3) Compute "manually" OLS estimate of regression parameters and error variance

# Regression matrix X and Response variable Y
Y = cars['dist'].values
X = np.column_stack((np.ones(len(Y)), cars['speed'].values))

# Compute OLS estimate of regression parameters
beta_ols = np.linalg.inv(X.T @ X) @ X.T @ Y

# Compute the residuals
residuals = Y - X @ beta_ols

# Compute the error variance
n, p = X.shape
error_variance = np.sum(residuals**2) / (n - p)

beta_ols, error_variance


In [ ]:
# Extracting Residual Standard Error (RSE) and Model Variance
RSE = np.sqrt(model_with_intercept.ssr / model_with_intercept.df_resid)
model_variance = RSE**2

RSE, model_variance

In [ ]:
!pip install rpy2
%load_ext rpy2.ipython

In [ ]:
%%R
summary(cars)

In [ ]:
%%R
# 3) Compute "manually" OLS estimate of regression parameters and error variance
# 4) Compute "manually" variance of estimated parameters

# simple linear model - simple regression without intercept
cars_lm1 <- lm(dist ~ -1 +speed , data = cars)
# simple linear model - simple regression with intercept
cars_lm2 <- lm(dist ~  speed , data = cars)


Y  = cars$dist
X1 = rep(1,times=length(Y))
X2 = cars$speed
X <- cbind(X1,X2)
X
# model  with intercept
# approach 1
beta_2_lm2 <- cor(X2,Y) * (sd(Y)/sd(X2))
beta_1_lm2 <- mean(Y) - beta_2_lm2*mean(X2)
cbind(beta_1_lm2,beta_2_lm2)
# approach 2
solve(t(X)%*%(X))%*%(t(X)%*%Y)
# approach 3 (from lm())
coefficients(cars_lm2)


# model  without intercept
solve(t(X2)%*%(X2))%*%(t(X2)%*%Y)
coefficients(cars_lm1)


#  variance of parameters
# manual computation of residuals
resid <-   Y - X%*%coefficients(cars_lm2)
summary(resid)
# same
residuals(cars_lm2)
sum(residuals(cars_lm2)==resid)

n <- length(Y)
p <-  2
sigma <- sqrt((1/(n-p))*sum(residuals(cars_lm2)^2))
summary(cars_lm2)
summary(cars_lm2)$sigma


# check
MSE  <- sum(resid^2)/(length(resid)) # MSE - biased MLE of residual variance
RMSE <- sqrt(MSE)                    # Root Mean Square Error (RMSE)

# IN Regression:
# The term MSE can be used to refer to the unbiased estimate of error variance too:
# Unbiased Pearson estimation of residual variance
sigma2  <- sum(resid^2)/(length(resid)-2)
sigma   <- sqrt(sigma2)


var_b2 <- sigma^2*(1/sum((X2 - mean(X2))^2))
sd_b2  <- sqrt(var_b2)
var_b2
sd_b2


var_b1  <- sigma^2*(sum(X2^2)/(n*sum((X2 - mean(X2))^2)))
sd_b1   <- sqrt(var_b1)
var_b1
sd_b1

# compare with a simple linear model - simple regression with intercept

summary(cars_lm2)$coefficients


### 4) Compute "manually" variance of estimated parameters


In [ ]:
# Compute the variance-covariance matrix of the estimated parameters
var_cov_matrix = error_variance * np.linalg.inv(X.T @ X)

# Extract variances of the parameters
var_beta = np.diagonal(var_cov_matrix)

# Extract standard errors of the parameters
se_beta = np.sqrt(var_beta)

var_beta, se_beta
# COMPARE WITH  model_with_intercept.summary()


### 5) Plot data with both estimated regression lines


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(cars['speed'], cars['dist'], color='black', label='Data')
plt.plot(cars['speed'], model_without_intercept.predict(cars['speed']), color='blue', label='Without Intercept')
plt.plot(cars['speed'], model_with_intercept.predict(cars['speed']), color='red', label='With Intercept')
plt.xlabel('Speed')
plt.ylabel('Stopping Distance')
plt.title('Speed vs Stopping Distance with Regression Lines')
plt.legend()
plt.grid(True)
plt.show()


### 6) Investigate output from the lm function
* Model Data: The first few rows of the original endogenous (or dependent) variable, dist, from the model.
* Fitted Values: The first few rows of the predicted values for dist based on the model.
* Residuals: The first few rows of the residuals, which represent the difference between the observed and fitted values.
* Degrees of Freedom of the Residuals:


In [ ]:
# Extracting various outputs from the model
model_data = model_with_intercept.model.data.orig_endog
fitted_values = model_with_intercept.fittedvalues
residuals = model_with_intercept.resid
df_residuals = model_with_intercept.df_resid

model_data.head(), fitted_values.head(), residuals.head(), df_residuals


### 7) Whats the estimated stopping distance for a car going 20mph?
* use both models  
* compare with the estimated stopping distance for a car going 30mph?
* use both models, can we predict this?

$\hat{\mu}$(dist) = -17.58 + 3.93(speed)

We estimated average stopping distance (in feet) for a car going (speed) mph.

Interpretation of the slope:

Estimated slope $\hat{\beta}_2$ = 3.93 represents the increase in average stopping distance for each mile per hour faster that the car drives.

Interpretation of the intercept:

Estimated intercept $\hat{\beta}_1$ = -17.58 represents the mean stopping distance for a car traveling 0 mph (make no sense in our example !!!)

Extrapolating can be dangerous and can lead to  nonsensical results.

In [ ]:
# Predicting for 20mph and 30mph for both models
speed_values = pd.DataFrame({'speed': [20, 30]})
predicted_without_intercept = model_without_intercept.predict(speed_values)
predicted_with_intercept = model_with_intercept.predict(speed_values)

predicted_values = pd.DataFrame({
    'Speed': [20, 30],
    'Without Intercept': predicted_without_intercept,
    'With Intercept': predicted_with_intercept
})

predicted_values


In [ ]:
from scipy.stats import t


In [ ]:
# We already have the data loaded and the model fitted, so we'll continue from the predictions

# t-test of significance of parameters (manually computed)
# H0 hypothesis is beta_i = 0
beta_H0 = 0
Sb_1 = model_with_intercept.bse[0]
UCI_1 = model_with_intercept.params[0] + t.ppf(0.975, df=n-2) * Sb_1
LCI_1 = model_with_intercept.params[0] + t.ppf(0.025, df=n-2) * Sb_1

Sb_2 = model_with_intercept.bse[1]
UCI_2 = model_with_intercept.params[1] + t.ppf(0.975, df=n-2) * Sb_2
LCI_2 = model_with_intercept.params[1] + t.ppf(0.025, df=n-2) * Sb_2

conf_int_manually = pd.DataFrame({
    'lower CI 2.5 %': [LCI_1, LCI_2],
    'estimated val.': [model_with_intercept.params[0], model_with_intercept.params[1]],
    'upper CI 97.5 %': [UCI_1, UCI_2]
}, index=['Intercept', 'Speed'])

# t-test of significance of parameters (manual computation)
tval_1_manual = (model_with_intercept.params[0] - beta_H0) / Sb_1
tval_2_manual = (model_with_intercept.params[1] - beta_H0) / Sb_2

pval_1_manual = 2 * (1 - t.cdf(np.abs(tval_1_manual), df=model_with_intercept.df_resid))
pval_2_manual = 2 * (1 - t.cdf(np.abs(tval_2_manual), df=model_with_intercept.df_resid))

coef_manually = pd.DataFrame({
    'Coefficient': model_with_intercept.params.values,
    'Standard Error': [Sb_1, Sb_2],
    't value': [tval_1_manual, tval_2_manual],
    'p value': [pval_1_manual, pval_2_manual]
}, index=['Intercept', 'Speed'])

results_manual = {
    'Confidence Intervals (manual computation)': conf_int_manually,
    'Coefficients (manual computation)': coef_manually
}

print(conf_int_manually)
print(coef_manually)


In [ ]:
t_values = model_with_intercept.tvalues
t_values

In [ ]:
p_values = model_with_intercept.pvalues
p_values

In [ ]:
%%R
# Mean Square Error and Standard Error
n = dim(cars)[1]
p = dim(cars)[2] - 1
MSE_2a = sum(residuals(cars_lm2)^2)/(n-p-1) # Mean square error  - manually from the definition
sigma_hat2a = sqrt(MSE_2a)                  # Residual standard error (estimation of standard deviation)
sigma_hat2b = summary(cars_lm2)$sigma       # by R function
cbind(sigma_hat2a, sigma_hat2b)
sigma_hat = sigma_hat2a
# compare to summary(cars_lm2)


# Interval Estimates of the Parameters
coef(cars_lm2)
confint(cars_lm2)
conf_int = as.data.frame(cbind(confint(cars_lm2)[,1], coef(cars_lm2), confint(cars_lm2)[,2]))
names(conf_int) = c("lower CI 2.5 %", "estimate", "upper CI 97.5 %"  )
conf_int
summary(cars_lm2)$coef
#summary(cars_lm2)$coefficients

# another confidence interval - 99%
confint(cars_lm2, level = 0.99)



# t-test of significance of parameters
Sb_1 = summary(cars_lm2)$coef[1,2]
UCI_1 = coef(cars_lm2)[1]+qt(.975,df = n-2)*Sb_1
LCI_1 = coef(cars_lm2)[1]+qt(.025, df = n-2)*Sb_1

Sb_2 = summary(cars_lm2)$coef[2,2]
UCI_2 = coef(cars_lm2)[2]+qt(.975, df = n-2)*Sb_2
LCI_2 = coef(cars_lm2)[2]+qt(.025, df = n-2)*Sb_2

conf_int_manually = rbind( c(LCI_1,coef(cars_lm2)[1],UCI_1), c(LCI_2,coef(cars_lm2)[2],UCI_2))
conf_int_manually = as.data.frame(conf_int_manually)
names(conf_int_manually) = c("lower CI 2.5 %", "estimateed val.", "upper CI 97.5 %"  )
row.names(conf_int_manually) = c("Intercept","Speed")
conf_int_manually

In [ ]:
%%R
# t-test of significance of parameters
# H0 hypothesis is beta_i = 0
beta_H0 = 0
tval_1 = (summary(cars_lm2)$coef[1,1]-beta_H0)/summary(cars_lm2)$coef[1,2]
tval_2 = (summary(cars_lm2)$coef[2,1]-beta_H0)/summary(cars_lm2)$coef[2,2]

pval_1 = 2*pt(abs(tval_1), cars_lm2$df.residual, lower.tail = FALSE)
pval_2 = 2*pt(abs(tval_2), cars_lm2$df.residual, lower.tail = FALSE)
coef_manually = rbind( c(coef(cars_lm2)[1], Sb_1, tval_1,pval_1),
                       c(coef(cars_lm2)[2], Sb_2, tval_2,pval_2))
coef_manually
#comapre with
summary(cars_lm2)$coef


In [ ]:

#  Confidence intervals vs. Prediction intervals

# Confidence intervals tell us about how well we have determined the mean.
# Prediction intervals tell us  where we can expect to see the next data point sampled.


### 8) Is the simple linear model good approximation for these problems? Suggest next steps.

